In [8]:
import pandas as pd
import numpy as np

In [40]:
data = pd.read_csv("data/loan_2007_2011.csv")

/Users/ajaykliyara/anaconda_py3/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
data.shape

(42536, 111)

In [42]:
data.head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [77]:
for i in data.columns:
    print(i)

id
member_id
loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
loan_status
pymnt_plan
url
desc
purpose
title
zip_code
addr_state
dti
delinq_2yrs
earliest_cr_line
inq_last_6mths
mths_since_last_delinq
mths_since_last_record
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
next_pymnt_d
last_credit_pull_d
collections_12_mths_ex_med
mths_since_last_major_derog
policy_code
application_type
annual_inc_joint
dti_joint
verification_status_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_il_6m
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
total_cu_tl
inq_last_12m
acc_open_past_24mths
avg_cur_bal
bc_open_to_buy
bc

In [48]:
#this verifies number of id's equal to number of records, therefore every row represents a
#unique customer
print(len(data['id'].unique()))
print(len(data['member_id'].unique()))


42536
42536


In [49]:
#loan_amnt != funded_amnt, therefore removing funded amnt
np.mean(data['loan_amnt'] != data['funded_amnt'])

0.046971976678578144

In [50]:
#emp title stands for designation of applier, high dimensional, leaving out for now
data['emp_title'].unique().shape

(30660,)

In [51]:
data['term'].unique()

array([' 36 months', ' 60 months', nan], dtype=object)

In [52]:
data.groupby('loan_status')['loan_status'].count()

loan_status
Charged Off                                             5655
Current                                                  445
Default                                                    1
Does not meet the credit policy. Status:Charged Off      761
Does not meet the credit policy. Status:Fully Paid      1988
Fully Paid                                             33663
In Grace Period                                            9
Late (16-30 days)                                          3
Late (31-120 days)                                        10
Name: loan_status, dtype: int64

In [ ]:
#may be just retain charged_off == bad customer, defaulted on loan
#fully paid = good customer

In [73]:
#all applications are individual , no joints therefore will drop all joint attributes.
data['application_type'].unique()

array(['INDIVIDUAL', nan], dtype=object)

In [75]:
data['mort_acc'].unique()

array([ nan])

In [76]:
#id,member_id - unique customer id's , does not contribute to ML models
field_deleted = ['id','member_id','issue_d','pymnt_plan','url','title','zip_code','addr_state',
                 'initial_list_status','out_prncp','out_prncp_inv','total_pymnt',
                 'total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee',
                 'recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt','next_pymnt_d',
                 'last_credit_pull_d','collections_12_mths_ex_med','policy_code','annual_inc_joint','dti_joint','verification_status_joint','open_il_6m','open_il_12m','open_il_24m','total_bal_il','open_rv_12m','open_rv_24m','total_cu_tl','mort_acc','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mths_since_recent_bc',
                 'mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m']
field_retained = ['loan_amnt','funded_amnt','funded_amnt_inv','term','int_rate','installment','grade','sub_grade','emp_length','home_ownership','annual_inc','verification_status','desc','purpose','dti','delinq_2yrs','earliest_cr_line','inq_last_6mths',
                  'mths_since_last_delinq','mths_since_last_record','open_acc','total_acc','revol_bal','revol_util','mths_since_last_major_derog','acc_now_delinq','tot_coll_amt','tot_cur_bal','mths_since_rcnt_il','il_util','max_bal_bc','all_util','total_rev_hi_lim','inq_fi','inq_last_12m','acc_open_past_24mths','avg_cur_bal','bc_open_to_buy',
                  'bc_util','chargeoff_within_12_mths','pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies','tax_liens','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit']

In [ ]:
#note - multiply installment by 12 and find ratio of annual_inc and may be use that as a predictor

In [ ]:
#desc - can be used for potential text analysis

In [ ]:
#zip code, addr state can be used for visualizing data on a map

In [ ]:
#dti dti joint can be correlated to income as it is dervied from it

In [ ]:
#out_prncp, out_prncp_inv not usefull when considering closed/default loans, so removing
#similarly for total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee

In [78]:
data_ss = data.loc[:,field_retained]

In [79]:
data_ss.shape

(42536, 48)

In [87]:
data_ss.to_csv?

In [88]:
data_ss.to_csv('data/loan_ss_2007_2011.csv',index=False)

In [89]:
data_ss_r = pd.read_csv('data/loan_ss_2007_2011.csv')

In [91]:
data_ss_r.shape

(42536, 48)

In [90]:
data_ss_r.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,bc_util,chargeoff_within_12_mths,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,10+ years,RENT,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,< 1 year,RENT,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,10+ years,RENT,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,10+ years,RENT,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,1 year,RENT,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
